In [5]:
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks as pn
import matplotlib.pyplot as plt
from pandapower.converter import to_ppc
from pandapower.pypower.makeYbus import makeYbus

pd.set_option('display.float_format', lambda x: '{:.4f}'.format(x))


# Crear la red a analizar
red2 = pp.create_empty_network()

# Creación de las barras del sistema
barra_1 = pp.create_bus(red2, vn_kv=110, name="Barra 1")
barra_2 = pp.create_bus(red2, vn_kv=220, name="Barra 2")
barra_1A = pp.create_bus(red2, vn_kv=220, name="Barra 1A")
barra_1B = pp.create_bus(red2, vn_kv=220, name="Barra 1B")
barra_2A = pp.create_bus(red2, vn_kv=220, name="Barra 2A")
barra_2B = pp.create_bus(red2, vn_kv=220, name="Barra 2B")
barra_3A = pp.create_bus(red2, vn_kv=220, name="Barra 3A")

# Añadir transformador entre barra 1 y 2
pp.create_transformer(red2, barra_2, barra_1, std_type="100 MVA 220/110 kV")

# Añadir el generador en la barra 1 y que dicha barra sea la referencia
pp.create_ext_grid(red2, barra_1, va_pu=1.0, name="Slack")

# Crear las líneas
pp.create_line(red2, barra_2, barra_1A, length_km=10.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1A")
pp.create_line(red2, barra_2, barra_1B, length_km=10.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1B")
pp.create_line(red2, barra_1A, barra_2A, length_km=15.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1A-2A")
pp.create_line(red2, barra_2A, barra_3A, length_km=20.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2A-3A")
pp.create_line(red2, barra_1B, barra_2B, length_km=30.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1B-2B")
pp.create_line(red2, barra_2B, barra_3A, length_km=15.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2B-3A")

# Crear las cargas
pp.create_load(red2, barra_1A, p_mw=30.0, q_mvar=20.0, name="Carga Barra 1A")
pp.create_load(red2, barra_2A, p_mw=52.5, q_mvar=35.0, name="Carga Barra 2A")
pp.create_load(red2, barra_3A, p_mw=22.5, q_mvar=15.0, name="Carga Barra 3A")
pp.create_load(red2, barra_1B, p_mw=15.0, q_mvar=10.0, name="Carga Barra 1B")
pp.create_load(red2, barra_2B, p_mw=90.0, q_mvar=60.0, name="Carga Barra 2B")

# Ejecutar el flujo de carga
pp.runpp(red2, algorithm='nr', numba=False, max_iteration=1000)

ppc = to_ppc(red2, init='flat')

# Obtener los datos de los buses y ramas
baseMVA = ppc["baseMVA"]
bus = ppc["bus"]
branch = ppc["branch"]
Vb=220e+3  # Tensión base V
Sb= 100e+6 # potencia base, modulo de 150M+100Mj
baseZ= Vb**2/Sb #impedancia base Ohm

# Crear la matriz Y-bus
Ybus, Yf, Yt = makeYbus(baseZ, bus, branch)

# Convertir la matriz Y-bus a formato denso
Ybus_dense = np.array(Ybus.todense())

# Convertir la matriz densa a una matriz de NumPy
Ybus_np = np.array(Ybus_dense)

# Aproximar los términos de la matriz a 4 decimales
Ybus_rounded = np.round(Ybus_np, decimals=4)

# Mostrar la matriz Y-bus densa en formato NumPy, aproximada a 3 decimales
print("Matriz Y-bus")
print(Ybus_rounded)

# Definir los parámetros y variables iniciales
V = np.ones(len(bus)-1)  # Vector de tensiones iniciales en pu
theta = np.zeros(len(bus)-1)  # Vector de ángulos de fase iniciales 
max_iter = 100 # Máximo número de iteraciones
tolerance = 1e-3  # Tolerancia para la convergencia
ne=[-1, -1, -1, -1, -1, -1]
neg = np.concatenate((ne,ne))
# Calcular P_inyec y Q_inyec
P_inyec = [0, -30e+6/Sb, -15e+6/Sb, -52.5e+6/Sb, -90e+6/Sb, -22.5e+6/Sb] # en pu
Q_inyec = [0, -20e+6/Sb, -10e+6/Sb, -35e+6/Sb, -60e+6/Sb, -15e+6/Sb] #en pu

# Función para calcular la potencia activa y reactiva inyectada en cada barra
def calc_potencias(V, theta, Ybus):
    n = len(V)
    P = np.zeros(n)
    Q = np.zeros(n)
    for i in range(n):
        for j in range(n):
            P[i] += V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
            Q[i] += V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
    return P, Q
    

def calc_jacobiano(V, theta, Ybus):
    n = len(V)
    Jaco = np.zeros((2*n, 2*n))
    #crear derivada parcial H
    aux = [0, 1, 2, 3, 4, 5]
    aux2 = [6, 7, 8, 9, 10, 11] 
    for i in aux:
        for j in aux:
            if i == j:
                Jaco[i, j] = -Q_inyec[i] - V[i]**2 * Ybus[i, i].imag
            else:
                Jaco[i, j] = V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
    #crear derivada parcial N
    for i in aux:
        for j in aux2:
            if i == j:
                Jaco[i, j] = P_inyec[i] / V[i] + V[i] * Ybus[i, i].real
            else:
                Jaco[i, j] = V[i] * (Ybus[i, j-6].real * np.cos(theta[i] - theta[j-6]) + Ybus[i, j-6].imag * np.sin(theta[i] - theta[j-6]))
    #crear derivada parcial M
    for i in aux2:
        for j in aux:
            if i == j:
                Jaco[i, j] = P_inyec[i-6] - V[i-6]**2 * Ybus[i-6, i-6].real
            else:
                Jaco[i, j] = - V[i-6] * V[j] * (Ybus[i-6, j].real * np.cos(theta[i-6] - theta[j]) + Ybus[i-6, j].imag * np.sin(theta[i-6] - theta[j]))
    #crear derivada parcial L
    for i in aux2:
        for j in aux2:
            if i == j:
                Jaco[i, j] = Q_inyec[i-6] / V[i-6] - V[i-6] * Ybus[i-6, i-6].imag
            else:
                Jaco[i, j] = V[i-6] * (Ybus[i-6, j-6].real * np.sin(theta[i-6] - theta[j-6]) - Ybus[i-6, j-6].imag * np.cos(theta[i-6] - theta[j-6]))
    return Jaco

# Método de Newton-Raphson
def newton_raphson(V, theta, Ybus, P_inyec, Q_inyec, tol=1e-3, max_iter=100):
    n = len(V)
    for i in range(max_iter):
        P, Q = calc_potencias(V, theta, Ybus)
        dP = (P_inyec - P)
        dQ = (Q_inyec - Q)
        dPQ = np.concatenate((dP, dQ))
        if np.linalg.norm(dPQ) < tol:
            print(f"Convergencia alcanzada en {i} iteraciones")
            break
        J = calc_jacobiano(V, theta, Ybus)
        J_T = np.transpose(J)#se transpone porque variables dPQ, V y theta estan en filas no columnas
        dX = np.linalg.solve(J_T, dPQ)
        theta += dX[:n]
        V += dX[n:]
    print(f"No se alcanzó la convergencia en {i} iteraciones")
    return V, theta

# Ejecutar Newton-Raphson
V, theta = newton_raphson(V, theta, Ybus, P_inyec, Q_inyec)

# Mostrar resultados de Newton-Raphson
print("Resultados de Newton-Raphson:")
print("Voltajes (V):", V)
print("Ángulos (theta):", theta)

# Mostrar resultados de pandapower
print("Resultados de pandapower:")
print(red2.res_bus)


Matriz Y-bus
[[ 1.80418000e+01  -833.1437j -1.80418000e+01  +833.1317j
   0.00000000e+00    +0.j      0.00000000e+00    +0.j
   0.00000000e+00    +0.j      0.00000000e+00    +0.j
   0.00000000e+00    +0.j    ]
 [-1.80418000e+01  +833.1317j  2.80908074e+04-45050.0102j
  -1.40363828e+04+22117.9365j -1.40363828e+04+22117.9365j
   0.00000000e+00    +0.j      0.00000000e+00    +0.j
   0.00000000e+00    +0.j    ]
 [ 0.00000000e+00    +0.j     -1.40363828e+04+22117.9365j
   2.33939714e+04-36839.4693j  0.00000000e+00    +0.j
  -9.35758850e+03+14745.291j   0.00000000e+00    +0.j
   0.00000000e+00    +0.j    ]
 [ 0.00000000e+00    +0.j     -1.40363828e+04+22117.9365j
   0.00000000e+00    +0.j      1.87151771e+04-29452.5688j
   0.00000000e+00    +0.j     -4.67879430e+03 +7372.6455j
   0.00000000e+00    +0.j    ]
 [ 0.00000000e+00    +0.j      0.00000000e+00    +0.j
  -9.35758850e+03+14745.291j   0.00000000e+00    +0.j
   1.63757799e+04-25770.9977j  0.00000000e+00    +0.j
  -7.01819140e+03+11058.9